In [47]:
from ctypes import CDLL
randlib = CDLL("libc.so.6")
import time, sys
import multiprocessing as mp

# Look here for spoilers of what I used to speed up code
import numpy as np
from numba import jit 
from numba.experimental import jitclass
from numba import float32, int32
from numba import types



def genrand3dpt(MIN, MAX):
    X = (MAX-MIN)*(float(randlib.rand())/2147483647)+MIN
    Y = (MAX-MIN)*(float(randlib.rand())/2147483647)+MIN
    Z = (MAX-MIN)*(float(randlib.rand())/2147483647)+MIN
    return X,Y,Z

class point:
    def __init__(self, x:float ,y: float, z: float):
        self.x, self.y, self.z = x, y, z
    
def setCase(case):
    if(case=='T1'):
        seed = 7
        pts = 40000
        cores = 1
        expected = "Expected Min, Max: 8.705  16830.027\n"
    elif case=='T2':
        seed = 7
        pts = 40000
        cores =2
        expected = "Expected Min, Max: 8.705  16830.027\n"
    elif case=='T3':
        seed = 7
        pts = 80000
        cores = 1
        expected = "Expected Min, Max: 0.260  16989.490\n"
    elif case=='T4':
        seed = 7
        pts = 80000
        cores = 2
        expected = "Expected Min, Max: 0.260  16989.490\n"
    elif case=='T5':
        seed = 7
        pts = 80000
        cores = 4
        expected = "Expected Min, Max: 0.260  16989.490\n"
    elif case=='T6':
        seed = 111
        pts = 111000
        cores = 3
        expected = "Expected Min, Max: 1.555  16965.943\n"
    return seed, pts, cores, expected

# Parallelization, Calling C and Other Methods: Part 2 of the Hit Series "Speeding Up My Code"

I had never done parallel coding in Python before this project, but I had done it in C in Dr. Butler's class. I needed to do some reading 

In [54]:
spec = [
    ('array', float32[:,:]),
    ('value', float32),
    ('value', float32),
    ('value', int32),
    ('value', int32),    
]


class args:
    def __init__(self, points, strt, cnt):
        self.pts=points
        self.MAX = 0
        self.MIN = 1.7976931348623157e+308
        self.start = strt
        self.ptcnt = cnt
        
@jit(types.Tuple((float32, float32))(float32[:,:],float32,float32,int32,int32))
def shortest_numba(pts, max, min, start, ptcnt):
    for i in range(start, ptcnt):
        for j in range(i+1, ptcnt):
            distance = (pts[i][0] - pts[j][0])**2 + (pts[i][1] - pts[j][1])**2 + (pts[i][2]- pts[j][2])**2
            if max < distance:
                max = distance
            elif min  > distance:
                min  = distance
    min, max = min **0.5, max**0.5
    print(max, min)    
    return max, min

In [55]:
%%time
seed, pts, corecnt, expected = setCase('T1')
randlib.srand(seed)
print(expected)
points = np.empty((pts,3), dtype=np.double)
for i in range(pts):
    points[i]  = genrand3dpt(0, 10000) # call random points macro   
test= args(points=points, strt=0, cnt=len(points))
l, m = shortest_numba(np.float32(test.pts), np.float32(test.MAX), np.float32(test.MIN), np.int32(test.start), np.int32(test.ptcnt))

Expected Min, Max: 8.705  16830.027

16830.026500276224 8.70447314793969
16830.02734375 8.704473495483398
CPU times: user 1.06 s, sys: 0 ns, total: 1.06 s
Wall time: 1.05 s
